In [12]:
from faker import Faker
import random
import time
import csv
import pandas as pd
import string 
import json 
faker = Faker()
# Load data from CSV
def load_csv(filename):
    return pd.read_csv(filename)
# Generate Unix timestamp
def generate_timestamp():
    return int(time.time())
# Write data to CSV
def write_to_csv(filename, fieldnames, data):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if file.tell() == 0:
            writer.writeheader()
        writer.writerows(data)
# Function to generate nullable values
def nullable_value(generator, null_probability=0.2):
    if random.random() < null_probability:
        return None
    return generator()
# Unique value generator
def unique_value(existing_set, generator):
    value = generator()
    while value in existing_set:
        value = generator()
    existing_set.add(value)
    return value



In [15]:
# Generate Airport data
#Bảng airport
current_airport_id = -1
def generate_airport_id():
    global current_airport_id
    current_airport_id += 1
    return current_airport_id
unique_iata = set()
unique_icao = set()
unique_airport_id = set()
location_data = load_csv('level_1/location.csv')
location_ids = list(location_data['LocationID'].unique())

# Hàm để lấy LocationID duy nhất
def get_unique_location_id():
    if not location_ids:
        raise ValueError("No more unique LocationIDs available.")
    return location_ids.pop(random.randrange(len(location_ids)))
def generate_airport(num_records):
    return [{
        'AirportID': unique_value(unique_airport_id, generate_airport_id),
        'IATACode': unique_value(unique_iata, lambda: faker.bothify(text='???').upper()),
        'ICAOCode': nullable_value(lambda: unique_value(unique_icao, lambda: faker.bothify(text='????').upper())),
        'Name': faker.company() + ' Airport',
        'LocationID': get_unique_location_id(),
        'CreatedAt': generate_timestamp(),
        'UpdatedAt': generate_timestamp()
    } for _ in range(num_records)]
num_records = int(input('Enter the number of records to generate: '))
# Generate and write data
write_to_csv('airports.csv', generate_airport(1)[0].keys(), generate_airport(num_records))

In [25]:
current_airlines_id = -1
def generate_airlines_id():
    global current_airlines_id
    current_airlines_id += 1
    return current_airlines_id
unique_icao_airline = set()
unique_iata = set()
def generate_airline(num_records):
    return [{
        'AirlineID': generate_airlines_id(),
        'IATACode': unique_value(unique_iata, lambda: faker.bothify(text='??').upper()),
        'ICAOCode': nullable_value(lambda: unique_value(unique_icao_airline, lambda: faker.bothify(text='????').upper())),
        'Name': faker.company() + ' Airlines',
        'Country': faker.country(),
        'CreatedAt': generate_timestamp(),
        'UpdatedAt': generate_timestamp()
    } for _ in range(num_records)]
num_records = int(input('Enter the number of records to generate: '))
# Generate and write data
write_to_csv('level_1/airlines.csv', generate_airline(1)[0].keys(), generate_airline(num_records))

In [67]:
domains = ['gmail.com', 'yahoo.com', 'outlook.com', 'hotmail.com', 'icloud.com']
passenger_type = ['ADT', 'CHD', 'INF']
current_passenger_id = -1
def generate_passenger_id():
    global current_passenger_id
    current_passenger_id += 1
    return current_passenger_id
unique_email = set()
unique_passenger_id = set()
def generate_passenger(num_records):
    return [{
        'PassengerID' : unique_value(unique_passenger_id, generate_passenger_id),
        'FirstName': faker.first_name(),
        'LastName': faker.last_name(),
        'DateOfBirth': faker.date_of_birth().strftime('%Y-%m-%d'),
        'Gender': random.choice(['Female', 'Male']),
        'Email': unique_value(unique_email, lambda: faker.user_name() + '@' + random.choice(domains)),
        'PhoneNumber':  faker.phone_number(),
        'PassportNumber':faker.bothify(text='?########').upper(),
        'Nationality': faker.country(),
        'PassengerType': random.choice(passenger_type),
        'CreatedAt': generate_timestamp(),
        'UpdatedAt': generate_timestamp()
    } for _ in range(num_records)]
num_records = int(input('Enter the number of records to generate: '))
# Generate and write data
write_to_csv('passenger.csv', generate_passenger(1)[0].keys(), generate_passenger(num_records))


In [10]:
from datetime import datetime, timedelta
today = datetime.today().date()
reasons = [
    "Flight control systems malfunction",
    "Engine or sensors not working",
    "Navigation or communication equipment failure",
    "Corrosion on airframe or wings",
    "Worn-out tires or brakes",
    "Missing maintenance logs",
    "Expired airworthiness certificate",
    "Maintenance not done as scheduled",
    "Unauthorized structural modifications",
    "Installation of non-compliant equipment",
    "Improperly repaired after an incident",
    "Operation in restricted airspace",
    "Violation of weight or fuel regulations",
    "Aircraft owned by unauthorized entity",
    "Use for illegal activities (e.g. smuggling)"
]

status = ["Active"] + reasons
weights = [0.95] + [0.05 / len(reasons)] * len(reasons)

def generate_registration(num_records):
    unique_registrations = set()

    def generate_unique_registration():
        while True:
            reg_num = faker.bothify(text='??-########').upper()
            if reg_num not in unique_registrations:
                unique_registrations.add(reg_num)
                return reg_num

    records = []
    for _ in range(num_records):
        registered_date = faker.date_between(start_date='-30y', end_date='today')
        arc_expired_date = registered_date + timedelta(days=365 + random.randint(-60, 60))

        record = {
            'RegistrationNumber': generate_unique_registration(),
            'ReferenceCapacity': random.randint(100, 500),
            'MWOT': random.randint(100, 400),
            'RegisteredDate': registered_date.strftime('%Y-%m-%d'),
            'ARCExpiredDate': arc_expired_date.strftime('%Y-%m-%d'),
            'RegistrationStatus': random.choices(status, weights=weights, k=1)[0]
        }
        records.append(record)

    return records
num_records = int(input('Enter the number of records to generate: '))

data = generate_registration(num_records)

write_to_csv('Registration.csv', data[0].keys(), data)

print(f"{num_records} records have been written to Registration.csv")


10000 records have been written to Registration.csv


In [68]:
# Danh sách mã ICAO, tên model và nhà sản xuất được cung cấp
aircraft_data = """
A124, Antonov An-124 Ruslan
A140, Antonov An-140
A148, Antonov An-148
A158, Antonov An-158
A19N, Airbus A319neo
A20N, Airbus A320neo
A21N, Airbus A321neo
A225, Antonov An-225 Mriya
A306, Airbus A300-600
A30B, Airbus A300B2, A300B4 and A300C4
A310, Airbus A310-200
A318, Airbus A318
A319, Airbus A319
A320, Airbus A320
A321, Airbus A321
A332, Airbus A330-200
A333, Airbus A330-300
A337, Airbus A330-700 "BelugaXL"
A338, Airbus A330-800
A339, Airbus A330-900
A342, Airbus A340-200
A343, Airbus A340-300
A345, Airbus A340-500
A346, Airbus A340-600
A359, Airbus A350-900
A35K, Airbus A350-1000
A388, Airbus A380-800
A3ST, Airbus A300-600ST "Super Transporter" / "Beluga"
A400, Airbus A400M Atlas
A748, Hawker Siddeley HS 748
AC90, Gulfstream/Rockwell (Aero) Turbo Commander 690
AJ27, Comac ARJ21-700 / C909
AN12, Antonov An-12
AN24, Antonov An-24
AN26, Antonov An-26
AN28, Antonov An-28
AN30, Antonov An-30
AN32, Antonov An-32
AN72, Antonov An-72 / An-74
AT43, Aerospatiale/Alenia ATR 42-300 / 320
AT45, Aerospatiale/Alenia ATR 42-500
AT46, Aerospatiale/Alenia ATR 42-600
AT72, Aerospatiale/Alenia ATR 72-201/-202
AT73, Aerospatiale/Alenia ATR 72-211/-212
AT75, Aerospatiale/Alenia ATR 72-212A (500)
AT76, Aerospatiale/Alenia ATR 72-212A (600)
ATP, British Aerospace ATP
B190, Beechcraft 1900
B37M, Boeing 737 MAX 7
B38M, Boeing 737 MAX 8
B39M, Boeing 737 MAX 9
B3XM, Boeing 737 MAX 10
"""

import csv
import random

# Hàm sinh sức chứa dựa vào nhà sản xuất
def generate_capacity(manufacturer):
    if manufacturer in ['Airbus', 'Boeing']:
        return random.randint(150, 400)
    elif manufacturer in ['Antonov', 'Lockheed']:
        return random.randint(50, 250)
    elif manufacturer in ['Comac', 'ATR', 'Beechcraft', 'Hawker']:
        return random.randint(30, 100)
    elif manufacturer in ['Gulfstream', 'Dassault', 'Cessna']:
        return random.randint(10, 30)
    else:
        return random.randint(20, 200)

# Dictionary để lưu kết quả
aircraft_lookup = {}

# Xử lý dữ liệu và đưa vào dictionary
for line in aircraft_data.strip().split("\n"):
    parts = line.split(", ")
    if len(parts) == 2:
        icao = parts[0].strip()
        model = parts[1].strip()
        manufacturer = model.split()[0]  # Lấy từ đầu tiên làm tên nhà sản xuất
        capacity = generate_capacity(manufacturer)  # Sinh sức chứa
        aircraft_lookup[icao] = (model, manufacturer, capacity)

# Ghi kết quả ra file CSV
csv_filename = "aircraft_lookup.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["ICAO", "Model", "Manufacturer", "Capacity"])
    for icao, (model, manufacturer, capacity) in aircraft_lookup.items():
        writer.writerow([icao, model, manufacturer, capacity])

csv_filename



'aircraft_lookup.csv'

In [11]:
# Load aircraft data from the uploaded CSV file
aircraft_data = load_csv('level_2/aircraft_lookup.csv')

# Convert the data into a list of tuples (ICAO, Model, Manufacturer)
aircraft_lookup = list(aircraft_data[['ICAO', 'Model', 'Manufacturer', 'Capacity']].itertuples(index=False, name=None))

In [12]:
registration_data = load_csv('level_2/Registration.csv')

# Convert the data into a list of tuples (ICAO, Model, Manufacturer)
registration_lookup = list(registration_data[['RegistrationNumber']].itertuples(index=False, name=None))

In [30]:
# Load aircraft data from the uploaded CSV file
airlinesID_data = load_csv('level_1/airlines.csv')

# Convert the data into a list of tuples (AirlinesID)
airlines_lookup = list(airlinesID_data[['AirlineID']].itertuples(index=False, name=None))

In [21]:
# Aircraft Table
current_aircraft_id = -1
def generate_aircraft_id():
    global current_aircraft_id
    current_aircraft_id += 1
    return current_aircraft_id
unique_aircraft_id = set()
unique_registration = set()
def generate_aircraft(num_records):
    return [{
        'AircraftID': unique_value(unique_aircraft_id, generate_aircraft_id),
        'RegistrationNumber': unique_value(unique_registration, lambda: (reg := random.choice(registration_lookup))[0]),
        'ICAOTypeCode': (icao := random.choice(aircraft_lookup))[0],
        'Model': icao[1], 
        'Manufacturer': icao[2],
        'OwnerAirlineID': (airlineid := random.choice(airlines_lookup))[0],
        'CreatedAt':  generate_timestamp(),
        'UpdatedAt':  generate_timestamp()
    } for _ in range(num_records)]
num_records = int(input('Enter the number of records to generate: '))
# Generate and write data
write_to_csv('level_2/aircraft.csv', generate_aircraft(1)[0].keys(), generate_aircraft(num_records))
# Passenger Table

In [24]:
unique_crew_member_id = set()
current_crewMember_id = -1
def generate_crewMember_id():
    global current_crewMember_id
    current_crewMember_id += 1
    return current_crewMember_id
def generate_crewMember(num_records):
    return [{
        'CrewMemberID': unique_value(unique_crew_member_id, generate_crewMember_id),
        'FirstName': faker.first_name(),
        'LastName': faker.last_name(),
        'Role': (role := random.choices(
            ['Captain', 'First Officer', 'Purser', 'Flight Attendant'],
            weights=[1, 2, 3, 4],  
            k=1 
        )[0]),
        'AirlineID': (airid := random.choice(airlines_lookup))[0], 
        'LicenseNumber': faker.bothify(text='LIC#########') if role in ['Captain', 'First Officer'] else None,
        'IsActive': random.choice([True, False]),
        'CreatedAt': generate_timestamp(),
        'UpdatedAt': generate_timestamp()
    } for _ in range(num_records)]
num_records = int(input('Enter the number of records to generate: '))
# Generate and write data
write_to_csv('level_2/crewMem.csv', generate_crewMember(1)[0].keys(), generate_crewMember(num_records))

In [31]:
# Load aircraft data from the uploaded CSV file
airportID_data = load_csv('level_1/airports.csv')

# Convert the data into a list of tuples (AirlinesID)
airports_lookup = list(airportID_data[['AirportID', 'IATACode']].itertuples(index=False, name=None))

In [46]:
unique_itinerary_id = set()
current_itinerary_id = 0

def generate_itinerary_id():
    global current_itinerary_id
    current_itinerary_id += 1
    return current_itinerary_id

def generate_itinerary(num_records):
    airport_id_to_iata = {a[0]: a[1] for a in airports_lookup}

    itineraries = []

    for _ in range(num_records):
        num_segments = random.randint(1, 4)

        route_airports = random.sample(airports_lookup, num_segments + 1)
        origin_id = route_airports[0][0]
        destination_id = route_airports[-1][0]
        via_ids = [ap[0] for ap in route_airports[1:-1]]

        origin_code = airport_id_to_iata[origin_id]
        destination_code = airport_id_to_iata[destination_id]
        via_codes = [airport_id_to_iata[vid] for vid in via_ids]

        if via_codes:
            itinerary_description = f"{origin_code}-{destination_code} via {', '.join(via_codes)}"
        else:
            itinerary_description = f"{origin_code}-{destination_code}"

        itineraries.append({
            'ItineraryID': unique_value(unique_itinerary_id, generate_itinerary_id),
            'OriginAirportID': origin_id,
            'DestinationAirportID': destination_id,
            'NumberOfSegments': num_segments,
            'ItineraryDescription': itinerary_description,
            'IsTemplate': random.choice([True, False]),
            'CreatedAt': generate_timestamp(),
            'UpdatedAt': generate_timestamp()
        })

    return itineraries
# Hàm ghi ra CSV
def write_to_csv(filename, fieldnames, data):
    with open(filename, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

# Gọi sinh dữ liệu và ghi ra file
num_records = int(input("Nhập số lượng hành trình cần sinh: "))
itinerary_data = generate_itinerary(num_records)
write_to_csv('level_2/itinerary.csv', itinerary_data[0].keys(), itinerary_data)
print("✅ Đã ghi dữ liệu hành trình ra file itinerary.csv")



ValueError: invalid literal for int() with base 10: ''

In [32]:
# Load aircraft data from the uploaded CSV file
airportID_data = load_csv('level_1/airports.csv')

# Convert the data into a list of tuples (AirlinesID)
airports_lookup = list(airportID_data[['AirportID']].itertuples(index=False, name=None))

In [33]:
from datetime import datetime, timedelta
flight_id_counter = 0
unique_flight_id = set()

def generate_unique_flight_id():
    global flight_id_counter
    flight_id_counter += 1
    return flight_id_counter

def generate_time():
    hour = random.randint(0, 23)
    minute = random.choice([0, 15, 30, 45])
    return f"{hour:02d}:{minute:02d}:00"

def generate_flight_number():
    return faker.bothify('??########').upper()
def generate_frequency():
    options = ['Daily', '1,3,5', '2,4,6', '6,7', '1,2,3,4,5', '1,3,4']
    return random.choice(options)


def generate_flights(num):
    flights = []
    for _ in range(num):
        dep, arr = random.sample(airports_lookup, 2)  # (AirportID, Code, Timezone)
        dep_time_str = generate_time()

        duration = random.randint(60, 1080)  

        flight_id = generate_unique_flight_id()
        frequency = generate_frequency()
        timestamp = int(datetime.utcnow().timestamp())

        flights.append({
            'FlightID': flight_id,
            'FlightNumber': generate_flight_number(),
            'AirlineID': random.choice(airlines_lookup)[0],
            'DepartureAirportID': dep[0],
            'ArrivalAirportID': arr[0],
            'ScheduledDepartureTime': dep_time_str, 
            'DurationMinutes': duration,
            'IsActive': random.choice([True, False]),
            'CreatedAt': timestamp,
            'UpdatedAt': timestamp,
            'Frequency': frequency
        })

    return flights

def generate_flight_operating_days(flight_records):
    operating_days = []
    for flight in flight_records:
        freq = flight['Frequency']
        days = list(range(1, 8)) if freq == 'Daily' else [int(d) for d in freq.split(',')]
        for d in days:
            operating_days.append({
                'FlightID': flight['FlightID'],
                'DayOfWeek': d,
                'CreatedAt': flight['CreatedAt'],
                'UpdatedAt': flight['UpdatedAt']
            })
    return operating_days

def write_to_csv(filename, fieldnames, data):
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

# ---- Chạy thử ----
num_flights = int(input("Nhập số lượng flight cần sinh: "))
flight_data = generate_flights(num_flights)
operating_day_data = generate_flight_operating_days(flight_data)

# Loại bỏ 'Frequency' trước khi ghi bảng Flight
for f in flight_data:
    f.pop('Frequency')

# Ghi file CSV
write_to_csv('level_2/flight.csv', flight_data[0].keys(), flight_data)
write_to_csv('level_2/flight_operating_day.csv', operating_day_data[0].keys(), operating_day_data)

print("✅ Đã ghi xong 2 file: flight.csv và flight_operating_day.csv")


C:\Users\huy37\AppData\Local\Temp\ipykernel_4652\880744236.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = int(datetime.utcnow().timestamp())


✅ Đã ghi xong 2 file: flight.csv và flight_operating_day.csv


In [47]:
# Load aircraft data from the uploaded CSV file
aircraftID_data = load_csv('aircraft.csv')

# Convert the data into a list of tuples (AircraftID)
aircraftID_lookup = list(aircraftID_data[['AircraftID']].itertuples(index=False, name=None))

In [45]:
# Hàm tạo dữ liệu AircraftSeat
seat_types = ['Window', 'Aisle', 'Middle', 'Exit Row', 'Bulkhead']
cabin_classes = ['Economy','Premium economy','Business class', 'First class']
characteristics = [
'Extra Legroom', 'Recline', 'Adjustable Headrest', 'Lumbar Support', 'Massage Function', 'Footrest',
'Power Outlet', 'USB Port', 'Personal TV', 'Wi-Fi Access', 'Audio Jack', 'Bluetooth Connectivity',
'In-seat Dining Table', 'Personal Reading Light', 'Coat Hook', 'Storage Pocket', 'Call Button', 
'Cup Holder', 'Adjustable Tray Table', 'Seatbelt Extender', 'Baby Bassinet Option', 
'Accessibility Assistance', 'Oxygen Mask', 'Privacy Divider', 'Noise-Cancelling Headphones', 
'Window Shade', 'Mood Lighting']
def generate_aircraft_seat(num_records):
    seats = []
    for _ in range(num_records):
        # Chọn các đặc điểm ngẫu nhiên
        seat_characteristics = random.sample(characteristics, random.randint(1, 5))
        
        seat = {
            'SeatID': unique_value(unique_seat_id, lambda: faker.bothify(text='ST########')),
            'AircraftID': random.choice(aircraftID_lookup)[0],
            'ConfigurationCode': faker.bothify(text='Y########'),
            'SeatNumber': f"{random.randint(1, 100)}{random.choice(['A', 'B', 'C', 'D', 'E', 'F'])}",
            'SeatType': random.choice(seat_types),
            'CabinClass': random.choice(cabin_classes),
            'Characteristics': json.dumps(seat_characteristics),  # Định dạng JSON đúng
            'IsBlocked': random.choices([True, False], weights=[10, 90], k=1)[0],
            'IsExitRow': random.choices([True, False], weights=[15, 85], k=1)[0],
            'CreatedAt': generate_timestamp(),
            'UpdatedAt': generate_timestamp()
        }
        seats.append(seat)
    return seats
    
# Số lượng bản ghi cần tạo
num_records = int(input('Enter the number of AircraftSeat records to generate: '))

# Ghi vào CSV
write_to_csv('aircraft_seat.csv', generate_aircraft_seat(1)[0].keys(), generate_aircraft_seat(num_records))


ValueError: invalid literal for int() with base 10: ''